In [120]:
! "c:\DATA SCIENCE\GEN AI\LangChain\Langchain Models\venv\Scripts\python.exe" -m pip install wikipedia



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [121]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
import wikipedia
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import TavilySearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool

In [122]:
load_dotenv()

# Initialize LLM
model = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5,
    max_tokens=512,
)

# Tavily search tool
search_tool = TavilySearchResults(max_results=3)

In [123]:
# wikipedia = WikipediaAPIWrapper(lang="en", top_k_results=1, doc_content_chars_max=100000, load_all_available_meta=True)
entity = input("Enter the entity you want to search -- ")

In [124]:
# Set language (optional)
wikipedia.set_lang("en")

# Get the full page object
page = wikipedia.page(entity, auto_suggest=False)

# Full article content
full_text = page.content

In [125]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50,
)

docs = splitter.create_documents([full_text])

In [126]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Create FAISS vector store from documents
vector_store = FAISS.from_documents(docs, embedding=embeddings)

# Save FAISS index locally
# vector_store.save_local("my_faiss_db")

# Load FAISS index back
vector_store = FAISS.load_local("my_faiss_db", embeddings, allow_dangerous_deserialization=True)

In [127]:
retriever = vector_store.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [128]:
# Wrap retriever into a Tool
wiki_tool = Tool(
    name="Wikipedia RAG",
    func=lambda q: "\n".join([doc.page_content for doc in retriever.invoke(q)]),
    description=f"Use this to answer questions about {entity} from Wikipedia."
)

In [129]:
# Hybrid Agent: Wikipedia RAG + Tavily Search
agent = initialize_agent(
    tools=[wiki_tool, search_tool],
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False
)

In [130]:
# Maintain chat history
messages = []

# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        print("Thank you, see you soon ")
        break
    print(f'You: {query}')
    # Append user message
    messages.append({"role": "user", "content": query})

    # Pass conversation history as context
    history_context = "\n".join(
        [f"{m['role'].capitalize()}: {m['content']}" for m in messages]
    )

    # Ask agent
    ans = agent.invoke(history_context)

    # Extract answer
    response = ans["output"]

    # Append AI message
    messages.append({"role": "assistant", "content": response})

    print(f"AI: {response}")

You: When was VK born
AI: Virat Kohli was born on November 5, 1988.
You: How many test matches he played
AI: 123 test matches.
Thank you, see you soon 
